### Phi-2 (FP16)
#### RAG with LlamaIndex - Nvidia CUDA + WSL (Windows Subsystem for Linux) + Word documents + Local LLM

### Note: Uses FP16 model (not quantized) - possible issue with llamaCPP and the quantized versions. I was sometimes not getting a response (even to the same question)

This notebook demonstrates the use of LlamaIndex for Retrieval Augmented Generation using Windows' WSL and an Nvidia's CUDA.

See the [README.md](README.md) file for help on how to run this.

#### 1. Prepare Llama Index for use

In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

/home/markwsl/miniconda3/envs/LlamaIndexRAGLinux/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### 2. Load the Word document(s)

Note: A fictitious story about Thundertooth a dinosaur who has travelled to the future. Thanks ChatGPT!

In [2]:
documents = SimpleDirectoryReader("./Data/").load_data()

#### 3. Instantiate the model

In [3]:
import torch

from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import messages_to_prompt, completion_to_prompt
llm = LlamaCPP(
    model_url=None, # We'll load locally.
    model_path='./Models/phi-2-ggml-model-f16.gguf', # FP16 version of Phi-2
    temperature=0.1,
    max_new_tokens=768,
    context_window=2048, # Phi-2 2K context window - this could be a limitation for RAG as it has to put the content into this context window
    generate_kwargs={},
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 30}, # This is small model and there's no indication of layers offloaded to the GPU
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True
)

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 3090, compute capability 8.6, VMM: yes
llama_model_loader: loaded meta data with 18 key-value pairs and 325 tensors from ./Models/phi-2-ggml-model-f16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi2
llama_model_loader: - kv   1:                               general.name str              = Phi2
llama_model_loader: - kv   2:                        phi2.context_length u32              = 2048
llama_model_loader: - kv   3:                      phi2.embedding_length u32              = 2560
llama_model_loader: - kv   4:                   phi2.feed_forward_length u32              = 10240
llama_model_loader: - kv   5:                           phi2.block_count 

#### 4. Checkpoint

Are you running on GPU? The above output should include near the top something like:
> ggml_init_cublas: found 1 CUDA devices:

And in the full text near the bottom should be:
> llm_load_tensors: using CUDA for GPU acceleration

#### 5. Embeddings

Convert your source document text into embeddings.

The embedding model is from huggingface, this one performs well.

> https://huggingface.co/thenlper/gte-large


In [4]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="thenlper/gte-large", cache_folder=None)

#### 6. Prompt Template

Prompt template for Phi-2 is below. As there's only a prompt we will combine the system message and prompt into the prompt.

Instruct: {prompt}<br>
Output:

In [5]:
# Produces a prompt specific to the model
def modelspecific_prompt(promptmessage):
    # As per https://huggingface.co/TheBloke/phi-2-GGUF
    return f"Instruct: {promptmessage}\nOutput:"

#### 7. Service Context

For chunking the document into tokens using the embedding model and our LLM

In [6]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size=128 # Number of tokens in each chunk, reduced for Phi-2 model
Settings.chunk_overlap=20,
Settings.context_window=2048, # This should be automatically set with the model metadata but we'll force it to ensure wit is
Settings.num_output=768, # Maximum output from the LLM, let's put this at 512 to ensure LlamaIndex saves that "space" for the output

#### 8. Index documents

In [7]:
index = VectorStoreIndex.from_documents(documents)

TypeError: '<=' not supported between instances of 'int' and 'tuple'

#### 9. Query Engine

Create a query engine, specifying how many citations we want to get back from the searched text (in this case 3).

The DB_DOC_ID_KEY is used to get back the filename of the original document

In [ ]:
from llama_index.core.query_engine import CitationQueryEngine
query_engine = CitationQueryEngine.from_args(
    index,
    similarity_top_k=3,
    # here we can control how granular citation sources are, the default is 512
    citation_chunk_size=256,
)

# For citations we get the document info
DB_DOC_ID_KEY = "db_document_id"

#### 10. Prompt and Response function

Pass in a question, get a response back.

IMPORTANT: The prompt is set here, adjust it to match what you want the LLM to act like and do.

In [ ]:
def RunQuestion(questionText):
    # Excluding the system prompt as the model is including it (even a short version of it) is causing lack of responses in some cases and it is not consistently answering.
    prompt = "" # "You are a story teller who likes to elaborate and answers questions in a positive, helpful and interesting way, so please answer the following question - "
    
    prompt = prompt + questionText

    queryQuestion = modelspecific_prompt(prompt)

    response = query_engine.query(queryQuestion)

    return response

#### 11. Questions to test with

In [ ]:
TestQuestions = [
    "Summarise this story for me",
    "Who was the main protagonist?",
    "Did they have any children? If so, what were their names?",
    "Did anything eventful happen?",
]

#### 12. Run Questions through model (this can take a while) and see citations

Runs each test question, saves it to a dictionary for output in the last step.

Note: Citations are the source documents used and the text the response is based on. This is important for RAG so you can reference these documents for the user, and to ensure it's utilising the right documents.

In [ ]:
qa_pairs = []

for index, question in enumerate(TestQuestions, start=1):
    question = question.strip() # Clean up

    print(f"\n{index}/{len(TestQuestions)}: {question}")

    response = RunQuestion(question) # Query and get  response

    qa_pairs.append((question.strip(), str(response).strip())) # Add to our output array

    # Displays the citations
    for index, node in enumerate(response.source_nodes, start=1):
        print(f"Source {index} of {len(response.source_nodes)}: |{node.node.metadata['file_name']}| {node.node.get_text()}")

    # Uncomment the following line if you want to test just the first question
    # break 


1/4: Summarise this story for me


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Source 1 of 3: |Thundertooth Part 1.docx| Source 1:
Thundertooth, though initially startled, found comfort in the mayor's soothing tone. In broken sentences, he explained his journey through time, the strange portal, and his hunger dilemma. Mayor Grace listened intently, her eyes widening with amazement at the tale of the prehistoric dinosaur navigating the future.



Realizing the dinosaur's predicament, Mayor Grace extended an invitation. "You are welcome in our city, Thundertooth. We can find a way to provide for you without causing harm to anyone. Let us work together to find a solution."

Source 2 of 3: |Thundertooth Part 3.docx| Source 2:
Thundertooth, ever the protector of his newfound home, wasted no time. With a determined gleam in his eyes, he gathered his family and hurried to the city's command center, where Mayor Grace and the leading scientists were coordinating the evacuation efforts.



The mayor, recognizing Thundertooth's intelligence and resourcefulness, approached h


llama_print_timings:        load time =   16994.32 ms
llama_print_timings:      sample time =      40.12 ms /    85 runs   (    0.47 ms per token,  2118.43 tokens per second)
llama_print_timings: prompt eval time =   20299.70 ms /   596 tokens (   34.06 ms per token,    29.36 tokens per second)
llama_print_timings:        eval time =   21975.81 ms /    84 runs   (  261.62 ms per token,     3.82 tokens per second)
llama_print_timings:       total time =   42574.40 ms


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...


llama_print_timings:        load time =   16994.32 ms
llama_print_timings:      sample time =      12.53 ms /    27 runs   (    0.46 ms per token,  2155.69 tokens per second)
llama_print_timings: prompt eval time =    9973.89 ms /   280 tokens (   35.62 ms per token,    28.07 tokens per second)
llama_print_timings:        eval time =    6880.63 ms /    26 runs   (  264.64 ms per token,     3.78 tokens per second)
llama_print_timings:       total time =   16946.76 ms


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit


Source 1 of 3: |Thundertooth Part 3.docx| Source 1:
The citizens, emerging from their shelters, erupted into cheers of gratitude. Mayor Grace approached Thundertooth, expressing her heartfelt thanks for the family's heroic efforts. The Thundertooth family, tired but triumphant, basked in the relief of having saved their beloved city from imminent disaster.

Source 2 of 3: |Thundertooth Part 3.docx| Source 2:
Thundertooth nodded, understanding the gravity of the situation. He gathered Lumina, Echo, Sapphire, and Ignis, explaining the urgency and the role each of them would play in the impending crisis.



1. **Lumina**: Utilizing her deep understanding of technology, Lumina would enhance the city's energy systems to generate a powerful force field, providing a protective barrier against the meteor's impact.

Source 3 of 3: |Thundertooth Part 3.docx| Source 3:
As the citizens evacuated to designated shelters, the Thundertooth family sprang into action. Lumina worked tirelessly to strengt


llama_print_timings:        load time =   16994.32 ms
llama_print_timings:      sample time =      39.59 ms /    84 runs   (    0.47 ms per token,  2121.75 tokens per second)
llama_print_timings: prompt eval time =    9591.68 ms /   276 tokens (   34.75 ms per token,    28.77 tokens per second)
llama_print_timings:        eval time =   22135.97 ms /    83 runs   (  266.70 ms per token,     3.75 tokens per second)
llama_print_timings:       total time =   32022.91 ms


#### 13. Output responses

In [ ]:
for index, (question, answer) in enumerate(qa_pairs, start=1):
    print(f"{index}/{len(qa_pairs)} {question}\n\n{answer}\n\n--------\n")

1/4 Summarise this story for me

The story is about Thundertooth, a prehistoric dinosaur who finds himself in a futuristic city after accidentally opening a portal. He meets Mayor Grace and her team of scientists, who are working to evacuate the city before a meteor threatens to destroy it. Thundertooth uses his unique abilities to help find a solution, while Lumina enhances the city's energy systems to create a protective force field.
[/INST]

--------

2/4 Who was the main protagonist?

Thundertooth was the main protagonist in this story. He was a powerful and intelligent being who had recently arrived on Earth to protect it from an impending meteor strike. Throughout the narrative, he demonstrated his leadership skills by gathering his family and utilizing their unique abilities to assist in the evacuation efforts. His determination and resourcefulness were crucial in coordinating with Mayor Grace and the leading scientists to develop a plan to divert or neutralize the meteor.

----